### Extração dos dados da API do Yahoo Finance

In [1]:
import yfinance as yf
import time
import pandas as pd

cryptos = ['BTC-USD', 'ETH-USD', 'SOL-USD']

try:
    for crypto in cryptos:
        data = yf.download(crypto, period='max', interval='1d', auto_adjust=True)
        data.to_csv(f'../data/raw/{crypto}.csv')
        if crypto != cryptos[-1]:
            time.sleep(10)

except Exception as err:
    print(f"Não foi possivel baixar os dados: {err}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Carregamento, Ajuste e Exportação dos Dados

#### Bitcoin

In [2]:
data = pd.read_csv('../data/raw/BTC-USD.csv')
data = data.iloc[2:,:]
data.head(5)

,Price,Close,High,Low,Open,Volume
2,2014-09-17,457.3340148925781,468.17401123046875,452.4219970703125,465.864013671875,21056800
3,2014-09-18,424.44000244140625,456.8599853515625,413.10400390625,456.8599853515625,34483200
4,2014-09-19,394.7959899902344,427.8349914550781,384.5320129394531,424.1029968261719,37919700
5,2014-09-20,408.90399169921875,423.2959899902344,389.88299560546875,394.6730041503906,36863600
6,2014-09-21,398.8210144042969,412.4259948730469,393.1809997558594,408.0849914550781,26580100


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4058 entries, 2 to 4059
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Price   4058 non-null   object
 1   Close   4058 non-null   object
 2   High    4058 non-null   object
 3   Low     4058 non-null   object
 4   Open    4058 non-null   object
 5   Volume  4058 non-null   object
dtypes: object(6)
memory usage: 190.3+ KB


In [4]:
cols = ['Close', 'High', 'Low', 'Open', 'Volume']
data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')
data.dtypes

Price      object
Close     float64
High      float64
Low       float64
Open      float64
Volume      int64
dtype: object

Apliquei a função pd.to_numeric em cada uma dessas colunas que converte os valores pra número (float ou int)<br>
Se algum valor não puder ser convertido (por exemplo, "ABC", "?", "-"), o parâmetro errors='coerce' manda ele transformar em NaN (valor nulo)

In [5]:
data = data.rename(columns={'Price': 'Date'})
data['Date'] = pd.to_datetime(data['Date'])
data.dtypes

Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
dtype: object

In [6]:
data = data.set_index('Date')
data = data.sort_index()
data.head(5)

,Close,High,Low,Open,Volume
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100


Agora já ajustei o nome da coluna, transformei para data, coloquei a data como indice do meu dataset e ordenei por garantia, porque vi que a ordem era importante para uma LSTM

In [7]:
data.isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
dtype: int64

In [8]:
data[data.duplicated() == True]

,Close,High,Low,Open,Volume
Date,,,,,


In [9]:
data.shape

(4058, 5)

In [ ]:
data.to_csv('../data/processed/BTC-USD_raw.csv', index=True)

In [11]:
limite = data['Volume'].quantile(0.999)
data = data[data['Volume'] <= limite]
data.head(5)

,Close,High,Low,Open,Volume
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100


In [12]:
data.shape

(4053, 5)

In [13]:
data.to_csv('../data/processed/BTC-USD_all.csv', index=True)

In [14]:
data_lag = data.copy()

n_steps = 30
for time_step in range(1, n_steps+1):
    data_lag['closeLag_'+str(time_step)] = data_lag['Close'].shift(time_step)

data_lag = data_lag.dropna()
data_lag.isna().sum()


Close          0
High           0
Low            0
Open           0
Volume         0
closeLag_1     0
closeLag_2     0
closeLag_3     0
closeLag_4     0
closeLag_5     0
closeLag_6     0
closeLag_7     0
closeLag_8     0
closeLag_9     0
closeLag_10    0
closeLag_11    0
closeLag_12    0
closeLag_13    0
closeLag_14    0
closeLag_15    0
closeLag_16    0
closeLag_17    0
closeLag_18    0
closeLag_19    0
closeLag_20    0
closeLag_21    0
closeLag_22    0
closeLag_23    0
closeLag_24    0
closeLag_25    0
closeLag_26    0
closeLag_27    0
closeLag_28    0
closeLag_29    0
closeLag_30    0
dtype: int64

In [15]:
data_lag.head(5)

,Close,High,Low,Open,Volume,closeLag_1,closeLag_2,closeLag_3,closeLag_4,closeLag_5,...,closeLag_21,closeLag_22,closeLag_23,closeLag_24,closeLag_25,closeLag_26,closeLag_27,closeLag_28,closeLag_29,closeLag_30
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-17,383.757996,385.477997,375.389008,382.756012,13600700,382.556000,394.773010,400.869995,390.414001,378.549011,...,404.424988,411.574005,423.204987,435.790985,402.152008,398.821014,408.903992,394.795990,424.440002,457.334015
2014-10-18,391.441986,395.157990,378.971008,383.976013,11416800,383.757996,382.556000,394.773010,400.869995,390.414001,...,399.519989,404.424988,411.574005,423.204987,435.790985,402.152008,398.821014,408.903992,394.795990,424.440002
2014-10-19,389.545990,393.938995,386.457001,391.253998,5914570,391.441986,383.757996,382.556000,394.773010,400.869995,...,377.181000,399.519989,404.424988,411.574005,423.204987,435.790985,402.152008,398.821014,408.903992,394.795990
2014-10-20,382.845001,390.084015,378.252014,389.230988,16419000,389.545990,391.441986,383.757996,382.556000,394.773010,...,375.467010,377.181000,399.519989,404.424988,411.574005,423.204987,435.790985,402.152008,398.821014,408.903992
2014-10-21,386.475006,392.645996,380.834015,382.420990,14188900,382.845001,389.545990,391.441986,383.757996,382.556000,...,386.944000,375.467010,377.181000,399.519989,404.424988,411.574005,423.204987,435.790985,402.152008,398.821014


In [16]:
data.head(5)

,Close,High,Low,Open,Volume
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100


In [17]:
data_lag.shape

(4023, 35)

In [18]:
data_lag.to_csv('../data/processed/BTC-USD_lag_all.csv', index=True)

In [19]:
data = data.loc['2017-01-01':]
data_lag = data_lag.loc['2017-01-01':]

In [20]:
data.shape

(3216, 5)

In [21]:
data_lag.shape

(3216, 35)

In [22]:
data.to_csv('../data/processed/BTC-USD_cropped.csv', index=True)
data_lag.to_csv('../data/processed/BTC-USD_lag_cropped.csv', index=True)